비어플 3주차 과제_백주원
=============
선형회귀 모형에 맞게 전처리하기 
-----------

[개요]
1. 범주형 데이터 수치화
2. 변수 선택
3. train,valid 셋 분리
4. 변수 스케일링

# 0-1 라이브러리 불러오기 

In [238]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler

# 0-2 데이터 불러오기

In [239]:
bicycle_2 = pd.read_csv("./bicycle_EDA.csv")

In [240]:
df1 = bicycle_2.copy()

In [241]:
# 모든 열 출력 설정
pd.set_option('display.max_columns', None)

In [242]:
df1

,Date,Rented Bike Count,Hour,Temperature(¡ÆC),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(¡ÆC),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,Year,Month,Day,day of week,Datetime,RR,Time Period,Temperature Range,DewPoint_cal,SS
0,2017-12-01,254,0,-5.2,37.0,2.200000,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 00:00:00,0,새벽 시간대,-10~0,-17.6,0
1,2017-12-01,204,1,-5.5,38.0,0.800000,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 01:00:00,0,새벽 시간대,-10~0,-17.5,0
2,2017-12-01,173,2,-6.0,39.0,1.000000,2000,-17.7,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 02:00:00,0,새벽 시간대,-10~0,-17.7,0
3,2017-12-01,107,3,-6.2,40.0,0.900000,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 03:00:00,0,새벽 시간대,-10~0,-17.6,0
4,2017-12-01,78,4,-6.0,36.0,1.169184,2000,-18.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 04:00:00,0,새벽 시간대,-10~0,-18.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,2018-11-30,1047,17,6.0,29.0,2.100000,1877,-10.7,0.23,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 17:00:00,0,저녁 시간대,0~10,-10.7,0
7004,2018-11-30,1384,18,4.7,34.0,1.900000,1661,-9.8,0.00,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 18:00:00,0,저녁 시간대,0~10,-9.8,0
7005,2018-11-30,1003,19,4.2,34.0,2.600000,1894,-10.3,0.00,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 19:00:00,0,저녁 시간대,0~10,-10.3,0
7006,2018-11-30,694,21,2.6,39.0,0.300000,1968,-9.9,0.00,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 21:00:00,0,저녁 시간대,0~10,-10.0,0


In [243]:
df1.columns = ['Date', 'Rent', 'Hour', 'Temp', 'Humid',
       'Windsp', 'Visibil', 'Dew point temp',
       'SolarRad', 'Rain', 'Snow', 'Seasons',
       'Holiday', 'Functioning Day', 'Year', 'Month', 'Day', 'day of week',
       'Datetime', 'RR', 'Time Period', 'Temperature Range', 'DewPoint_cal','SS']

In [244]:
df1 = df1.drop(['DewPoint_cal'],axis=1)
# EDA 때 사용했던 파생변수 제거

In [245]:
df1 = df1 [df1['Functioning Day'] != 'No' ]  
# 운행 안 하는 날 행 제거 

In [246]:
df1

,Date,Rent,Hour,Temp,Humid,Windsp,Visibil,Dew point temp,SolarRad,Rain,Snow,Seasons,Holiday,Functioning Day,Year,Month,Day,day of week,Datetime,RR,Time Period,Temperature Range,SS
0,2017-12-01,254,0,-5.2,37.0,2.200000,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 00:00:00,0,새벽 시간대,-10~0,0
1,2017-12-01,204,1,-5.5,38.0,0.800000,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 01:00:00,0,새벽 시간대,-10~0,0
2,2017-12-01,173,2,-6.0,39.0,1.000000,2000,-17.7,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 02:00:00,0,새벽 시간대,-10~0,0
3,2017-12-01,107,3,-6.2,40.0,0.900000,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 03:00:00,0,새벽 시간대,-10~0,0
4,2017-12-01,78,4,-6.0,36.0,1.169184,2000,-18.6,0.00,0.0,0.0,Winter,No Holiday,Yes,2017,12,1,4,2017-12-01 04:00:00,0,새벽 시간대,-10~0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,2018-11-30,1047,17,6.0,29.0,2.100000,1877,-10.7,0.23,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 17:00:00,0,저녁 시간대,0~10,0
7004,2018-11-30,1384,18,4.7,34.0,1.900000,1661,-9.8,0.00,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 18:00:00,0,저녁 시간대,0~10,0
7005,2018-11-30,1003,19,4.2,34.0,2.600000,1894,-10.3,0.00,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 19:00:00,0,저녁 시간대,0~10,0
7006,2018-11-30,694,21,2.6,39.0,0.300000,1968,-9.9,0.00,0.0,0.0,Autumn,No Holiday,Yes,2018,11,30,4,2018-11-30 21:00:00,0,저녁 시간대,0~10,0


In [247]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6772 entries, 0 to 7007
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               6772 non-null   object 
 1   Rent               6772 non-null   int64  
 2   Hour               6772 non-null   int64  
 3   Temp               6772 non-null   float64
 4   Humid              6772 non-null   float64
 5   Windsp             6772 non-null   float64
 6   Visibil            6772 non-null   int64  
 7   Dew point temp     6772 non-null   float64
 8   SolarRad           6772 non-null   float64
 9   Rain               6772 non-null   float64
 10  Snow               6772 non-null   float64
 11  Seasons            6772 non-null   object 
 12  Holiday            6772 non-null   object 
 13  Functioning Day    6772 non-null   object 
 14  Year               6772 non-null   int64  
 15  Month              6772 non-null   int64  
 16  Day                6772 non-n

In [248]:
df1.isna().sum()

Date                 0
Rent                 0
Hour                 0
Temp                 0
Humid                0
Windsp               0
Visibil              0
Dew point temp       0
SolarRad             0
Rain                 0
Snow                 0
Seasons              0
Holiday              0
Functioning Day      0
Year                 0
Month                0
Day                  0
day of week          0
Datetime             0
RR                   0
Time Period          0
Temperature Range    0
SS                   0
dtype: int64

In [249]:
df2 = df1.copy()

# 0-3 변수 정리

|변수명|의미|변수 타입|
|---|---|---|
|__*Rent(종속변수)*__|대여 자전거 수|수치형|
|Date|날짜 (일/월/년도)|범주형|
|Hour|시간|범주형|
|Seasons|계절|범주형|
|Holiday|휴일|범주형|
|Functioning Day|운영일|범주형|
|Temp|기온|수치형|
|Humid|상대습도|수치형|
|Windsp|풍속|수치형|
|Visibil|가시성|수치형|
|Dew point temp|이슬점|수치형|
|SolarRad|태양 복사량|수치형|
|Rain|강수량|수치형|
|Snow|눈의 양|수치형|
|__파생변수__||
|Datetime|년도-월-일 00:00:00|범주형|
|day of week|요일|범주형|
|Year|년도|범주형|
|Month|월|범주형|
|Day|일|범주형|
|Time Period|시간대|범주형|
|Temperature Range|기온 구간|범주형|
|RR|강수 여부|범주형|
|SS|강설 여부|범주형|

\* 모델링에 사용할 변수 : 시간대,계절,휴일|기온,상대습도,풍속,가시성,이슬점,태양복사량,강수량,강설량
<br>\* 범주형 변수의 값이 너무 많은 것은 구간을 나눈 파생변수를 만들어 사용

- 범주형 데이터 
  - 명목형 변수 : 계절,휴일,시간대,강수여부,강설여부 (시간변수는 명목형)
  - 순서형 변수 : X
- 수치형 데이터 : 기온,상대습도,풍속,가시성,이슬점,태양복사량

# 1. 범주형 변수 수치화

## 명목형 변수 One-Hot Encoding

방법1) pd.get_dummies 이용
- 인코딩 결과를 바로 데이터프레임으로 변환해서 전처리 작업에 편리
- 전처리 단계에서 독립적으로 사용되며, 머신러닝 파이프라인과 직접적으로 통합되지는 않음

In [250]:
df1_onehot1 = pd.get_dummies(df1, columns=['Seasons','Holiday','Time Period','RR','SS'], drop_first=True)
# 변수 당 열 하나 삭제해도 잃는 정보는 없음
# 변수 간의 상관관계 감소시켜 다중공선성 문제 완화

In [251]:
df1_onehot1

,Date,Rent,Hour,Temp,Humid,Windsp,Visibil,Dew point temp,SolarRad,Rain,Snow,Functioning Day,Year,Month,Day,day of week,Datetime,Temperature Range,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday_No Holiday,Time Period_새벽 시간대,Time Period_아침 시간대,Time Period_저녁 시간대,RR_1,SS_1
0,2017-12-01,254,0,-5.2,37.0,2.200000,2000,-17.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 00:00:00,-10~0,False,False,True,True,True,False,False,False,False
1,2017-12-01,204,1,-5.5,38.0,0.800000,2000,-17.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 01:00:00,-10~0,False,False,True,True,True,False,False,False,False
2,2017-12-01,173,2,-6.0,39.0,1.000000,2000,-17.7,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 02:00:00,-10~0,False,False,True,True,True,False,False,False,False
3,2017-12-01,107,3,-6.2,40.0,0.900000,2000,-17.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 03:00:00,-10~0,False,False,True,True,True,False,False,False,False
4,2017-12-01,78,4,-6.0,36.0,1.169184,2000,-18.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 04:00:00,-10~0,False,False,True,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,2018-11-30,1047,17,6.0,29.0,2.100000,1877,-10.7,0.23,0.0,0.0,Yes,2018,11,30,4,2018-11-30 17:00:00,0~10,False,False,False,True,False,False,True,False,False
7004,2018-11-30,1384,18,4.7,34.0,1.900000,1661,-9.8,0.00,0.0,0.0,Yes,2018,11,30,4,2018-11-30 18:00:00,0~10,False,False,False,True,False,False,True,False,False
7005,2018-11-30,1003,19,4.2,34.0,2.600000,1894,-10.3,0.00,0.0,0.0,Yes,2018,11,30,4,2018-11-30 19:00:00,0~10,False,False,False,True,False,False,True,False,False
7006,2018-11-30,694,21,2.6,39.0,0.300000,1968,-9.9,0.00,0.0,0.0,Yes,2018,11,30,4,2018-11-30 21:00:00,0~10,False,False,False,True,False,False,True,False,False


방법2) sklearn의 OneHotEncoder 이용
- 기본적으로 희소 행렬(sparse matrix)을 반환함
- 희소행렬 : 메모리 효율 높음. 큰 데이터셋에 유리
- fit_transform 후에 데이터프레임으로 변환해야함
- 머신러닝 모델링 파이프라인에서 반복적이고 일관된 전처리 작업을 필요로 할 때 적합함
- `방법2 채택` 인코딩의 목적이 모델링이기 때문에 방법2로 채택

In [252]:
# OneHotEncoder 객체 생성
encoder = OneHotEncoder(drop='first', sparse_output=False) 
## drop='first'로 다중공선성 방지, sparse_output=False로 희소행렬이 아닌 밀집행렬로 반환

# 'Seasons','Holiday' 열을 원핫 인코딩
encoded_columns = encoder.fit_transform(df2[['Seasons', 'Holiday','Time Period','RR','SS']])

In [253]:
encoder.categories_

[array(['Autumn', 'Spring', 'Summer', 'Winter'], dtype=object),
 array(['Holiday', 'No Holiday'], dtype=object),
 array(['낮 시간대', '새벽 시간대', '아침 시간대', '저녁 시간대'], dtype=object),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64)]

In [254]:
# 인코딩된 데이터를 데이터프레임으로 변환
encoded_df2 = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['Seasons', 'Holiday','Time Period','RR','SS']))

In [255]:
encoded_df2

,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday_No Holiday,Time Period_새벽 시간대,Time Period_아침 시간대,Time Period_저녁 시간대,RR_1,SS_1
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
6767,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6768,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6769,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6770,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [256]:
# 인덱스 맞추기
df2 = df2.reset_index(drop=True)

In [257]:
df2.drop(['Seasons', 'Holiday','Time Period','RR','SS'], axis=1)

,Date,Rent,Hour,Temp,Humid,Windsp,Visibil,Dew point temp,SolarRad,Rain,Snow,Functioning Day,Year,Month,Day,day of week,Datetime,Temperature Range
0,2017-12-01,254,0,-5.2,37.0,2.200000,2000,-17.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 00:00:00,-10~0
1,2017-12-01,204,1,-5.5,38.0,0.800000,2000,-17.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 01:00:00,-10~0
2,2017-12-01,173,2,-6.0,39.0,1.000000,2000,-17.7,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 02:00:00,-10~0
3,2017-12-01,107,3,-6.2,40.0,0.900000,2000,-17.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 03:00:00,-10~0
4,2017-12-01,78,4,-6.0,36.0,1.169184,2000,-18.6,0.00,0.0,0.0,Yes,2017,12,1,4,2017-12-01 04:00:00,-10~0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6767,2018-11-30,1047,17,6.0,29.0,2.100000,1877,-10.7,0.23,0.0,0.0,Yes,2018,11,30,4,2018-11-30 17:00:00,0~10
6768,2018-11-30,1384,18,4.7,34.0,1.900000,1661,-9.8,0.00,0.0,0.0,Yes,2018,11,30,4,2018-11-30 18:00:00,0~10
6769,2018-11-30,1003,19,4.2,34.0,2.600000,1894,-10.3,0.00,0.0,0.0,Yes,2018,11,30,4,2018-11-30 19:00:00,0~10
6770,2018-11-30,694,21,2.6,39.0,0.300000,1968,-9.9,0.00,0.0,0.0,Yes,2018,11,30,4,2018-11-30 21:00:00,0~10


In [258]:
df2_onehot2 = pd.concat([df2.drop(['Seasons', 'Holiday','Time Period','RR','SS'], axis=1), encoded_df2],axis =1)

In [259]:
df3 = df2_onehot2

## 순서형 변수 Label Encoding

In [260]:
'''from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoders = {}

# 각 열에 대해 Label Encoding 수행
for column in [' ']:
    encoder = LabelEncoder()
    df3[column] = encoder.fit_transform(df3[column])
    label_encoders[column] = encoder'''

"from sklearn.preprocessing import LabelEncoder\n\n# LabelEncoder 객체 생성\nlabel_encoders = {}\n\n# 각 열에 대해 Label Encoding 수행\nfor column in [' ']:\n    encoder = LabelEncoder()\n    df3[column] = encoder.fit_transform(df3[column])\n    label_encoders[column] = encoder"

# 2. 변수 선택

In [262]:
df3.columns

Index(['Date', 'Rent', 'Hour', 'Temp', 'Humid', 'Windsp', 'Visibil',
       'Dew point temp', 'SolarRad', 'Rain', 'Snow', 'Functioning Day', 'Year',
       'Month', 'Day', 'day of week', 'Datetime', 'Temperature Range',
       'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter',
       'Holiday_No Holiday', 'Time Period_새벽 시간대', 'Time Period_아침 시간대',
       'Time Period_저녁 시간대', 'RR_1', 'SS_1'],
      dtype='object')

In [263]:
df4 = df3.drop(['Date', 'Hour','Rain', 'Snow', 'Functioning Day', 'Year','Month', 'Day',
                 'day of week', 'Datetime', 'Temperature Range'],axis=1)

In [264]:
df4

,Rent,Temp,Humid,Windsp,Visibil,Dew point temp,SolarRad,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday_No Holiday,Time Period_새벽 시간대,Time Period_아침 시간대,Time Period_저녁 시간대,RR_1,SS_1
0,254,-5.2,37.0,2.200000,2000,-17.6,0.00,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,204,-5.5,38.0,0.800000,2000,-17.6,0.00,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,173,-6.0,39.0,1.000000,2000,-17.7,0.00,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,107,-6.2,40.0,0.900000,2000,-17.6,0.00,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,78,-6.0,36.0,1.169184,2000,-18.6,0.00,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6767,1047,6.0,29.0,2.100000,1877,-10.7,0.23,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6768,1384,4.7,34.0,1.900000,1661,-9.8,0.00,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6769,1003,4.2,34.0,2.600000,1894,-10.3,0.00,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6770,694,2.6,39.0,0.300000,1968,-9.9,0.00,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


# 3. train, valid 셋 분리

In [265]:
#X(독립변수), y(종속변수) 분리
X = df4.drop("Rent", axis=1)
y = df4["Rent"]

In [267]:
#train과 valid 셋 분리

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=100)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(5417, 15) (1355, 15) (5417,) (1355,)


In [269]:
X_train

,Temp,Humid,Windsp,Visibil,Dew point temp,SolarRad,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday_No Holiday,Time Period_새벽 시간대,Time Period_아침 시간대,Time Period_저녁 시간대,RR_1,SS_1
1222,1.1,26.0,2.100000,1957,-16.3,0.82,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4365,28.0,69.0,1.699545,1140,21.7,0.00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
337,0.5,76.0,0.900000,364,-3.2,0.17,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1035,-10.5,27.0,4.000000,1959,-26.3,1.26,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
818,-8.2,29.0,2.000000,1920,-23.1,1.19,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6026,20.0,22.0,1.600000,1950,-2.3,2.29,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
79,-5.3,45.0,2.500000,2000,-15.3,0.00,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
3927,21.2,87.0,0.800000,376,18.9,0.03,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5955,14.8,68.0,2.000000,1502,8.9,0.00,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [270]:
X_val

,Temp,Humid,Windsp,Visibil,Dew point temp,SolarRad,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday_No Holiday,Time Period_새벽 시간대,Time Period_아침 시간대,Time Period_저녁 시간대,RR_1,SS_1
115,1.200,94.0,0.600000,228,0.3,0.00,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1147,-8.211,42.0,2.100000,1981,-19.3,0.00,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
6021,7.800,70.0,1.400000,1034,2.6,0.29,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1751,2.100,97.0,0.200000,859,1.6,0.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2096,8.000,47.0,3.300000,2000,-2.6,0.00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,22.500,96.0,0.300000,1605,21.8,0.00,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
6350,11.800,86.0,1.600000,1819,9.5,0.00,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
743,-1.900,69.0,5.000000,791,-6.8,0.00,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4521,32.300,63.0,2.100000,1037,24.3,0.41,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


# 4. 변수 스케일링

[선형 회귀 모형]
- 통계학적 가정 :  
  1. 수집된 관측값이 정규분포를 이룬다.
  2. 종속변수와 독립변수 간의 관계가 선형이다
  3. 독립변수 간에 상관관계가 없다
  4. 오차항은 평균이 0인 정규분포를 이룬다.
  5. 오차항을 모든 독립변수의 값에 대해 동일한 분산을 갖는다. 
- 머신러닝 기반 회귀: 통계적 가정보다는 데이터 기반의 패턴 학습에 중점을 두며, 통계적 가정이 만족되지 않아도 모델이 유용하게 동작할 수 있다.
- 하지만, 이상치에 민감하고 변수(수치형)의 스케일링 필요함 


In [271]:
Ss = StandardScaler()

X_train_Ss = Ss.fit_transform(X_train)
X_val_Ss = Ss.transform(X_val)

print("<train 셋>")
print(f'평균: {X_train_Ss.mean()}\n분산: {X_train_Ss.std()}')
print("<valid 셋>")
print(f'평균: {X_val_Ss.mean()}\n분산: {X_val_Ss.std()}')

<train 셋>
평균: 4.1580588376583305e-17
분산: 1.0
<valid 셋>
평균: -0.0003011987266107005
분산: 1.000924288910286


In [272]:
# 모델링 전 최종 데이터셋

X_train_Ss
X_val_Ss

y_train
y_val

115      190
1147      81
6021     784
1751     147
2096     731
        ... 
5131     218
6350      78
743      107
4521    1994
3087    2410
Name: Rent, Length: 1355, dtype: int64